# Readability assessment

## Description

We compute accuracy for BERT + Average Noun per phrase on One Stop English Dataset using 10-folds cross validation

In [19]:
!pip install transformers
!pip install datasets

In [44]:
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer,BertModel, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from collections import defaultdict

from sklearn.metrics import confusion_matrix, classification_report,precision_score,accuracy_score,f1_score
from sklearn.model_selection import StratifiedKFold

import logging
logging.basicConfig(level = logging.ERROR)


from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

In [21]:
import warnings
warnings.filterwarnings("ignore")

In [22]:
from datasets import load_dataset

dataset = load_dataset("onestop_english")


In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 567
    })
})

In [25]:
import pandas as pd

df = pd.DataFrame(dataset['train'])
df.columns = ['content', 'labels']


In [26]:
df

,content,labels
0,"When you see the word Amazon, what’s the first...",0
1,"To tourists, Amsterdam still seems very libera...",0
2,"Anitta, a music star from Brazil, has millions...",0
3,Google has made maps of the world’s highest mo...,0
4,The auction of a Banksy painting that disappea...,0
...,...,...
562,"In typical bad-boyfriend style, Dan Sullivan w...",2
563,Thousands of people protested on Australia’s b...,2
564,1 Race engineer \nA race engineer liaises betw...,2
565,More than one million British workers might be...,2


In [27]:
df.labels.value_counts()


0    189
1    189
2    189
Name: labels, dtype: int64

In [28]:
num_diff = df.labels.unique()
diff = ['beginner','intermediate','advanced']

label = dict(zip(num_diff, diff))

diff = list(diff)


In [29]:
df['Difficulty'] = df.labels.replace(label)
df.head()



,content,labels,Difficulty
0,"When you see the word Amazon, what’s the first...",0,beginner
1,"To tourists, Amsterdam still seems very libera...",0,beginner
2,"Anitta, a music star from Brazil, has millions...",0,beginner
3,Google has made maps of the world’s highest mo...,0,beginner
4,The auction of a Banksy painting that disappea...,0,beginner


Data Preprocessing

In [30]:
import re

In [31]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [32]:

def clean_text(text):
    #text=' '.join([contraction_mapping[i] if i in contraction_mapping.keys() else i for i in text.split()])
    text=re.sub("'s","",text)
    if text.split()[0] == 'Intermediate':
      text = ' '.join(text.split()[1:])
    text=' '.join([i for i in text.split() if i.isalpha()])
    text=re.sub('[^a-zA-Z]'," ",text)


    return text

In [33]:
df['content'] = df['content'].apply(clean_text)


In [34]:
df[df['Difficulty']=='intermediate']


,content,labels,Difficulty
189,When you see the word whats the first thing yo...,1,intermediate
190,To Amsterdam still seems very Recently the cit...,1,intermediate
191,Brazils latest funk has won millions of fans b...,1,intermediate
192,It has mapped the worlds highest the ocean the...,1,intermediate
193,The controversial auction of a Banksy mural th...,1,intermediate
...,...,...,...
373,In typical Dan Sullivan was late to breakfast ...,1,intermediate
374,Thousands of people protested on Australias be...,1,intermediate
375,Race engineer A race engineer liaises between ...,1,intermediate
376,More than one million British workers might be...,1,intermediate


In [35]:
df

,content,labels,Difficulty
0,When you see the word the first thing you thin...,0,beginner
1,To Amsterdam still seems very Recently the May...,0,beginner
2,a music star from has millions of but she is a...,0,beginner
3,Google has made maps of the highest the ocean ...,0,beginner
4,The auction of a Banksy painting that disappea...,0,beginner
...,...,...,...
562,In typical Dan Sullivan was late to breakfast ...,2,advanced
563,Thousands of people protested on beaches again...,2,advanced
564,Race engineer A race engineer liaises between ...,2,advanced
565,More than one million British workers might be...,2,advanced


sentence lenth

In [36]:
!pip install lingfeat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 MB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 933.2/933.2 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 15.4 MB/s eta 0:00:00
  Created wheel for lingfeat: filename=lingfeat-1.0.0b19-py3-none-any.whl size=71127048 sha256=1782ee0131b1c8dfece031f0314ec613debc3ff74373508cbcbd8d07ed47bff9
  Stored in directory: /root/.cache/pip/wheels/f0/6c/25/7cc4b4dbfc7c605bad9f8824afef1b4abc8ed18232972c86a8
Successfully built lingfeat


In [37]:
from lingfeat import extractor

Downloading: https://github.com/yzhangcs/parser/releases/download/v1.1.0/ptb.crf.con.lstm.char.zip to /root/.cache/supar/ptb.crf.con.lstm.char.zip
100%|██████████| 334M/334M [00:02<00:00, 155MB/s]


In [38]:
def get_avg_nounphrase(text):
  LingFeat = extractor.pass_text(text)
  LingFeat.preprocess()
  avg = LingFeat.PhrF_()['as_NoPhr_C']
  return avg

In [45]:
df['avg_nounphrases'] = df['content'].progress_apply(get_avg_nounphrase)

  0%|          | 0/567 [00:00<?, ?it/s]

In [46]:
df

,content,labels,Difficulty,avg_nounphrases
0,When you see the word the first thing you thin...,0,beginner,48.00
1,To Amsterdam still seems very Recently the May...,0,beginner,62.50
2,a music star from has millions of but she is a...,0,beginner,37.75
3,Google has made maps of the highest the ocean ...,0,beginner,41.75
4,The auction of a Banksy painting that disappea...,0,beginner,36.00
...,...,...,...,...
562,In typical Dan Sullivan was late to breakfast ...,2,advanced,36.60
563,Thousands of people protested on beaches again...,2,advanced,72.00
564,Race engineer A race engineer liaises between ...,2,advanced,125.00
565,More than one million British workers might be...,2,advanced,71.50


In [47]:
from transformers import BertTokenizer,BertModel, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup


In [48]:
class DiffDataset(Dataset):
    def __init__(self, contents, labels, features, tokenizer, max_len):
        self.contents = contents
        self.labels = labels
        self.tokenizer = tokenizer
        self.features = features
        self.max_len = max_len

    def __len__(self):
        return len(self.contents)

    def __getitem__(self, item):
        content = str(self.contents[item])
        labels = self.labels[item]
        features = self.features[item]
        # Tokenizing the texts, while also including special tokens
        # for start and end of the text, as well as padding
        encoding = self.tokenizer.encode_plus(
          content,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt', # We return here the data as Pytorch Tensor
        )

        return {
          'content': content,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(labels, dtype=torch.long),
          'features': torch.tensor([features], dtype=torch.long)
        }


In [49]:
torch.tensor([[12, 23, 45]], dtype=torch.long).shape

torch.Size([1, 3])

In [ ]:
# df_temp, df_test = train_test_split(df, test_size=0.2, random_state=42)
# df_train, df_val = train_test_split(df_temp, test_size=0.1, random_state=42)

# print("size of training set : " + str(df_train.shape[0]))
# print("size of validation set : " + str(df_val.shape[0]))
# print("size of test set : " + str(df_test.shape[0]))



In [64]:
def generate_dataloader(df, tokenizer, max_len, batch_size):
    ds = DiffDataset(
        contents=df.content.to_numpy(),
        labels=df.labels.to_numpy(),
        features = df.avg_nounphrases.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len,
      )
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=2
      )


In [51]:
#model_name = 'bert-base-multilingual-uncased'
model_name = 'bert-base-uncased'

In [52]:
tokenizer = BertTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [53]:
max_len = 256
batch_size = 8

# train_dataloader = generate_dataloader(df_train, tokenizer, max_len,batch_size)
# val_dataloader = generate_dataloader(df_val, tokenizer, max_len,batch_size)
# test_dataloader = generate_dataloader(df_test, tokenizer, max_len,batch_size)

#data = next(iter(train_dataloader))



In [54]:
class DiffClassifier(nn.Module):

    def __init__(self, n_classes):
        super(DiffClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        # Adding drop out, keeping 90% of the last neurons of the raw BERT model
        #self.drop = nn.Dropout(p=0.1)
        # The last linear layer for multiclass classification
        self.out = nn.Linear(self.bert.config.hidden_size+1, n_classes)

    # Forward propagation function
    def forward(self, input_ids, attention_mask, features):
        model_outs = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        last_hidden_state = model_outs[0]
        pooled_output = model_outs[1]

        x = torch.cat((pooled_output, features) , dim=1)

        return self.out(x)


In [55]:
len(diff)

3

In [56]:
# Computing multiclass precision for the outputs of the model
def compute_precision(outputs, labels):
  op = outputs.cpu()
  la = labels.cpu()
  _, preds = torch.max(op, dim=1)
  # We choose 'weighted' averaging of the precision of each label because it takes into account the imbalance of labels in our tweets dataset
  # other viable averaging methods are 'micro'
  return torch.tensor(precision_score(la, preds, average='weighted',zero_division=0))

In [57]:
def train(model, train_dataloader,optimizer,scheduler):

        # Reset tracking variables at the beginning of each epoch
        precision, correct_predictions, batch_counts = 0, 0, 0
        losses = []

        # Put the model into the training mode
        model.train()


        # For each batch of training data...
        for d in train_dataloader:
              batch_counts +=1
              # Load batch to GPU
              input_ids = d["input_ids"].to(device)
              attention_mask = d["attention_mask"].to(device)
              labels = d["labels"].to(device)
              features = d["features"].to(device)

              outputs = model(input_ids=input_ids,attention_mask=attention_mask, features=features)

              _, preds = torch.max(outputs, dim=1)

              # Compute loss and accumulate the loss values

              loss = loss_fn(outputs, labels)

              correct_predictions += torch.sum(preds == labels)
              losses.append(loss.item())
              precision +=  compute_precision(outputs, labels)

              loss.backward()

              # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
              torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
              # Update parameters and the learning rate
              optimizer.step()
              scheduler.step()
              optimizer.zero_grad()
          # Accuracy, loss, precision
        return correct_predictions.double() / len(df_train), np.mean(losses), precision/batch_counts

In [58]:
def eval(model, valid_dataloader, loss_fn, device, n):
    model = model.eval()


    correct_predictions , precision ,batch_counts = 0,0,0
    losses = []

    with torch.no_grad():
        for d in valid_dataloader:
            batch_counts += 1

            # Preparing inputs
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)
            features = d["features"].to(device)

            # Running inference using the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                features=features
              )

            # Running softmax on the outputs
            _, preds = torch.max(outputs, dim=1)

            # Computing loss function
            loss = loss_fn(outputs, labels)

            # Counting the correct occurences
            correct_predictions += torch.sum(preds == labels)

            # Computing the precision (true positives/true positives + false positives)
            # for each class and label, and find their average weighted by support
            precision += compute_precision(outputs,labels)

            losses.append(loss.item())
    # Accuracy, loss, precision
    return correct_predictions.double()/n, np.mean(losses), precision/batch_counts

In [59]:
def testing(model, dataloader):
  model = model.eval()

  contents = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in dataloader:

      texts = d["content"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      labels = d["labels"].to(device)
      features = d["features"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        features=features
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      contents.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(labels)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return contents, predictions, prediction_probs, real_values

In [60]:
best_model_path = "./bert_model_baseline.bin"
def save_best_model(model):
  torch.save(model.state_dict(), best_model_path)

def load_best_model(model):
  model.load_state_dict(torch.load(best_model_path))
  return model

In [61]:
class EarlyStopping:
    def __init__(self, model, patience=1):
        self.patience = patience  # number of times to allow for no improvement before stopping the execution
        self.counter = 0  # count the number of times the validation accuracy not improving
        self.min_validation_loss = np.inf
        self.model = model
        self.max_acc = 0

    # return True when validation loss is not decreased by the `min_delta` for `patience` times
    def early_stop_check(self, val_acc):
        if (val_acc > self.max_acc):
            self.max_acc = val_acc
            self.counter = 0  # reset the counter if validation loss decreased at least by min_delta
            save_best_model(model)
        else:
            self.counter += 1 # increase the counter if validation loss is not decreased by the min_delta
            if self.counter >= self.patience:
                return True
        return False

In [62]:
df

,content,labels,Difficulty,avg_nounphrases
0,When you see the word the first thing you thin...,0,beginner,48.00
1,To Amsterdam still seems very Recently the May...,0,beginner,62.50
2,a music star from has millions of but she is a...,0,beginner,37.75
3,Google has made maps of the highest the ocean ...,0,beginner,41.75
4,The auction of a Banksy painting that disappea...,0,beginner,36.00
...,...,...,...,...
562,In typical Dan Sullivan was late to breakfast ...,2,advanced,36.60
563,Thousands of people protested on beaches again...,2,advanced,72.00
564,Race engineer A race engineer liaises between ...,2,advanced,125.00
565,More than one million British workers might be...,2,advanced,71.50


In [65]:
b_accuracy = 0
epochs = 10
n_fold = 10
random_state = 42
folds_acc = []
histories = []


k_fold = StratifiedKFold(n_splits=n_fold, random_state=random_state, shuffle=True)
y = df.drop(["content", "Difficulty", "avg_nounphrases"], axis=1)

for i, (temp_index, test_index) in enumerate(k_fold.split(df, y)):
    print(" ------   Fold {}  ------- ".format(i+1), end="\n")
    df_temp = df.iloc[temp_index]
    df_train, df_val = train_test_split(df_temp, test_size=0.2, random_state=42)
    df_test = df.iloc[test_index]
    train_dataloader = generate_dataloader(df_train, tokenizer, max_len,batch_size)
    val_dataloader = generate_dataloader(df_val, tokenizer, max_len,batch_size)
    test_dataloader = generate_dataloader(df_test, tokenizer, max_len,batch_size)


    data = next(iter(train_dataloader))
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    features = data['features'].to(device)

    model = DiffClassifier(len(diff))
    # Running the classifier on GPU
    model = model.to(device)

    optimizer = AdamW(model.parameters(),
                  lr=2e-5,
                  weight_decay = 0.2,
                  correct_bias=False)
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0.1,
                                                num_training_steps=len(train_dataloader)*epochs)
    loss_fn = nn.CrossEntropyLoss().to(device)

    history = defaultdict(list)

    early_stopping = EarlyStopping(model, patience=4)

    for epoch in range(epochs):

      print(f'Epoch {epoch + 1}/{epochs}')
      print('-' * 20)

      train_acc, train_loss, train_preci = train(
            model,
            train_dataloader,
            optimizer,
            scheduler,
          )
      print(f"Train : Loss {train_loss}, Accuracy : {train_acc*100:.2f} %, Precision : {train_preci}")

      history['train_acc'].append(train_acc)
      history['train_loss'].append(train_loss)
      history['train_precision'].append(train_preci)

      val_acc, val_loss, val_preci = eval(
            model,
            val_dataloader,
            loss_fn,
            device,
            len(df_val),
          )

      print(f'Val : Loss :{val_loss}, Accuracy : {val_acc*100:.2f} %, Precision : {val_preci}')
      print()

      history['val_acc'].append(val_acc)
      history['val_loss'].append(val_loss)
      history['val_precision'].append(val_preci)

      histories.append(history)

      if early_stopping.early_stop_check(val_acc):
        break

      # if val_acc > b_accuracy:
      #   torch.save(model.state_dict(), best_model_path)
      #   b_accuracy = val_acc

    model = load_best_model(model)
    y_contents, y_pred, y_pred_probs, y_test = testing(
          model,
          test_dataloader
        )
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy for fold {}: {:.2f}%\n".format(i+1, acc*100))
    folds_acc.append(acc)


 ------   Fold 1  ------- 


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1/10
--------------------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Train : Loss 0.02505947442670517, Accuracy : 99.51 %, Precision : 0.9967320261437909


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Val : Loss :0.990042641830559, Accuracy : 80.58 %, Precision : 0.8351648351648352



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Test Accuracy for fold 10: 83.93%



In [66]:
print("The accuracy score is: {:.3f}%".format(sum(folds_acc) * 100 / len(folds_acc)))

The accuracy score is: 86.942%


In [ ]:
model.load_state_dict(torch.load("best_bert_model_baseline.bin"))

y_contents, y_pred, y_pred_probs, y_test = testing(
  model,
  test_dataloader
)

out_df = pd.DataFrame()
out_df['content'] = y_contents
out_df['true_label'] = y_test
out_df['pred_label'] = y_pred

In [ ]:
out_df

In [ ]:
out_df.to_csv('out_BERT_256_simple features.csv')